# Instrument control
We will connect to a fake instrument and take data, then FFT the data to extract a signal. First, we will import `fake_pyvisa`, which will act like `pyvisa` without needing to connect to a physical instrument. We will also import `numpy`, `matplotlib.pyplot`, and our simple fft function.

In [ ]:
import fake_pyvisa
import numpy as np 
import matplotlib.pyplot as plt
from simple_fft import simple_fft

Create the resource manager instance. We will connect to the instrument via LAN. The IP address is '192.168.2.125'. Create the instrument instance and query the device ID to confirm the connection.

In [ ]:
rm = fake_pyvisa.ResourceManager()

In [ ]:
ip_address = '192.168.2.125'
inst = rm.open_resource(f'TCPIP0::{ip_address}::INSTR') 
inst.query('*IDN?')

We expect our signal to be have a range between 1 V and 10 V, so we need to set the voltage range to 10 V, rather than the default 1 V. From the programming manual, we find that the command syntax is 'C[channel]:VOLT_DIV [range]V', where channel is the channel index and range is the voltage range. Set the voltage range on the first channel to 10 V.

In [ ]:
inst.write('C1:VOLT_DIV 10V')

From the programming manual, we find that the command to get the sample rate is 'SARA?'. Query the sample rate, and create the variable 'tsample' in s.

In [ ]:
response = inst.query('SARA?')
tsample = float(response.replace('SARA ', '').replace('Sa/s\n', ''))

From the programming manual, we find that the command to recieve data is 'C[channel]:WF? DAT2'. Query the device for data and create the variable 'x'. Create the corresponding time array 'time'.

In [ ]:
x = inst.query('C1:WF? DAT2')
time = np.arange(0, len(x) * tsample, tsample)

Plot the signal timestream. Don't forget the axis labels.

In [ ]:
plt.plot(time, x)

FFT the data and plot the result. Can you extract the signal? If not, try taking data again. The noise in this system is on the edge of our ability to extract the peak, so we may find that the peak is visible in some datasets and not others.

In [ ]:
f, y = simple_fft(tsample, x)
f, y = f[1:], y[1:]

In [ ]:
plt.plot(f, y)
ix = np.argmax(y)
plt.plot(f[ix], y[ix], marker = 'x', color = 'black')
print(f[ix])

### Averaging